In [15]:
import cv2
import numpy as np
from scipy.fft import fft
from skimage.feature import local_binary_pattern
import torch
import torch.nn as nn
from torchvision import transforms, models
from torchcam.methods import GradCAM  
import matplotlib.pyplot as plt

def get_activation_map(model, input_tensor, target_layer='layer4'):
    # Store the gradients
    gradients = []
    def save_gradient(grad):
        gradients.append(grad)
    
    # Get the output of the target layer
    for name, module in model.cnn.named_children():
        input_tensor = module(input_tensor)
        if name == target_layer:
            input_tensor.register_hook(save_gradient)
            layer_output = input_tensor
    
    # Get model output
    output = model(input_tensor)
    
    # Calculate gradients
    if len(gradients) > 0:
        # Get the gradient for the target class
        output.backward()
        gradients = gradients[0].cpu().data.numpy()
        weights = np.mean(gradients, axis=(2, 3))[0, :]
        
        # Generate heatmap
        cam = np.zeros(layer_output.shape[2:], dtype=np.float32)
        for i, w in enumerate(weights):
            cam += w * layer_output[0, i].cpu().data.numpy()
            
        cam = np.maximum(cam, 0)
        cam = (cam - np.min(cam)) / (np.max(cam) - np.min(cam) + 1e-7)
        return cam
    return None
# ----------------------
# 1. INITIALIZATION
# ----------------------
cap = cv2.VideoCapture(0)
background_subtractor = cv2.createBackgroundSubtractorMOG2()
intensity_history = []

# Load HSV reference image
hsv_reference = cv2.imread("hsv-0.png") or np.zeros((100, 300, 3), dtype=np.uint8)

# ----------------------
# 2. FIRE DETECTION FUNCTIONS
# ----------------------
def fire_color_mask(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # Adjusted HSV range for fire-like colors (red, orange, yellow)
    lower_bound1 = np.array([0, 50, 50], dtype=np.uint8)   # Red hues
    upper_bound1 = np.array([10, 255, 255], dtype=np.uint8)
    lower_bound2 = np.array([160, 50, 50], dtype=np.uint8) # Red hues (wrap-around)
    upper_bound2 = np.array([179, 255, 255], dtype=np.uint8)
    lower_bound3 = np.array([18, 100, 100], dtype=np.uint8) # Orange-yellow hues
    upper_bound3 = np.array([35, 255, 255], dtype=np.uint8)
    
    # Combine masks for red and orange-yellow hues
    mask1 = cv2.inRange(hsv, lower_bound1, upper_bound1)
    mask2 = cv2.inRange(hsv, lower_bound2, upper_bound2)
    mask3 = cv2.inRange(hsv, lower_bound3, upper_bound3)
    mask = cv2.bitwise_or(mask1, mask2)
    mask = cv2.bitwise_or(mask, mask3)
    
    # Morphological operations to clean up the mask
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    # Find contours and filter based on size and shape
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) > 1000:  # Increased area threshold for better accuracy
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = float(w) / h
            if 0.5 < aspect_ratio < 2.0:  # Filter based on aspect ratio to avoid false positives
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                cv2.putText(frame, "Fire Detected!", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    return frame, mask

    mask = cv2.bitwise_or(
        cv2.inRange(hsv, lower1, upper1),
        cv2.inRange(hsv, lower2, upper2)
    )
    mask = cv2.bitwise_or(mask, cv2.inRange(hsv, lower3, upper3))
    
    kernel = np.ones((5,5), np.uint8)
    return cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

def detect_motion(frame):
    fg_mask = background_subtractor.apply(frame)
    return cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8))

def check_flicker(history, sample_rate=30):
    if len(history) < sample_rate: return False
    yf = fft(history[-sample_rate:])
    dominant_freq = np.abs(np.fft.fftfreq(sample_rate, 1/sample_rate)[np.argmax(np.abs(yf))])
    return 8 < dominant_freq < 12  # Fire flickers at ~10Hz

def analyze_texture(roi_gray):
    lbp = local_binary_pattern(roi_gray, 24, 3, method='uniform')
    return np.histogram(lbp, bins=np.arange(27), density=True)[0]

# ----------------------
# 3. DEEP LEARNING MODEL
# ----------------------
class FireDetector(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = nn.Identity()
        self.lstm = nn.LSTM(512, 128, batch_first=True)
        self.classifier = nn.Linear(128, 1)

    def forward(self, x):
        batch_size, seq_len = x.shape[0], x.shape[1]
        features = self.cnn(x.view(-1, *x.shape[2:]))
        _, (hidden, _) = self.lstm(features.view(batch_size, seq_len, -1))
        return torch.sigmoid(self.classifier(hidden[-1]))

# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FireDetector().to(device)
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ----------------------
# 4. MAIN PROCESSING LOOP
# ----------------------
frame_buffer = []
while True:
    ret, frame = cap.read()
    if not ret: break
    
    # Traditional Computer Vision
    mask = fire_color_mask(frame)
    motion_mask = detect_motion(frame)
    gray = cv2.cvtColor(cv2.bitwise_and(frame, frame, mask=mask), cv2.COLOR_BGR2GRAY)
    
    # Feature Extraction
    intensity_history.append(np.mean(gray) if np.any(gray) else 0)
    is_flickering = check_flicker(intensity_history)
    texture_feat = analyze_texture(gray)
    
    # Deep Learning
    frame_buffer.append(transform(frame))
    if len(frame_buffer) == 30:  # Process every 1 sec (30fps)
        with torch.no_grad():
            prob = model(torch.stack(frame_buffer).unsqueeze(0).to(device)).item()
        frame_buffer = []
    
    # Decision Fusion
    cv_decision = (np.sum(motion_mask) > 1000 and is_flickering and np.sum(mask) > 5000)
    dl_decision = prob > 0.7 if len(frame_buffer)==0 else False
    
    if cv_decision or dl_decision:
        x,y,w,h = cv2.boundingRect(cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0][0])
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255), 2)
        cv2.putText(frame, "FIRE DETECTED", (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
    
    # Display
    cv2.imshow("Live", frame)
    cv2.imshow("Fire Mask", mask)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'torchcam'

In [18]:
python -c "import torchcam; print(torchcam.__version__)"

SyntaxError: invalid syntax (356571872.py, line 1)